# Master Thesis: Bankruptcy Prediction for European Countries
Code written by Marc Zeugin (UZH)

## Load modules

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, PrecisionRecallDisplay, confusion_matrix, RocCurveDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.experimental import enable_iterative_imputer
from sklearnex.ensemble import RandomForestClassifier
from sklearnex.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import IterativeImputer
from xgboost.sklearn import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
from skopt.space import Real, Integer
from matplotlib import pyplot as plt
from skopt import BayesSearchCV
from joblib import dump, Memory
from sklearnex.svm import SVC
from itertools import product
from tempfile import mkdtemp
from shutil import rmtree
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import warnings
warnings.filterwarnings(action="ignore")
import pyarrow
import glob
import time

Set random seed

In [ ]:
np.random.seed(1)

## 1. Setup key variables

In [ ]:
# determine training/testing ratio, default is 0.2
tt_size = 0.2
# number of splits for k-fold crossvalidation, default is 5
k_splits = 5
# set number of jobs to run in parallel, -1 means all processors, default is 1
jobs = -1
# number of iterations for BayesSearchCV
n_iterations = 40
# select scoring model to optimize, default is average_precision, the value used for the precision-recall curve
scoring_metric = 'average_precision'
# label for classification report
label = ['Non-Bankrupt', 'Bankrupt']

# absolute path to dataset
path = 'C:/Users/marczeugin/Documents/Masterthesis/datasets/'
# extension of dataset type
ext = '*.csv'

# determine the size (in inches) of the precision-recall curve figure, default is (4, 2)
figure_size = (4, 2)
# set dpi of small graphs, default is 100
dpi_low = 100
# set dpi of medium graphs, default is 200
dpi_med = 200
# set dpi of large graphs, default is 250
dpi_high = 250

# enable subsample of total dataset to be used for hyperparameter tuning, default is True
allow_subsample = True
# set subsample size of total dataset, default is .1 e.g. 10%
subsample_size = 0.1
# enable quick overview to run with subsample
allow_subsample_overview = True
# enable hyperparameter tuning with subsample
allow_subsample_hyperparameter = True

# allow to load dataset with SMOTEENN already run and imputation already imputed, default is True
allow_computed_set = True

Create model lists

In [ ]:
name_list = ['complete', 'without years', 'without macro', 'without years and macro', 'complete SMOTEENN', 'without years SMOTEENN', 'without macro SMOTEENN', 'without years and macro SMOTEENN']

X_list = np.load('X_list.npy', allow_pickle=True).tolist()
X_list_m = np.load('X_list_m.npy', allow_pickle=True).tolist()
y_list = np.load('y_list.npy', allow_pickle=True).tolist()
y_list_m = np.load('y_list_m.npy', allow_pickle=True).tolist()

### 6.9. Hyperparameter tuning for XGB

#### 6.9.1. Without SMOTEENN

In [ ]:
params_list_xgb = dict()
params_list_xgb['n_estimators'] = Integer(10, 3500)
params_list_xgb['max_depth'] = Integer(2, 25)
params_list_xgb['gamma'] = Real(0, 0.9)
model = XGBClassifier(random_state=1, n_jobs=jobs, tree_method='hist')
kfold = StratifiedKFold(n_splits=k_splits, random_state=1, shuffle=True)
grid = BayesSearchCV(estimator=model, search_spaces=params_list_xgb, scoring=scoring_metric, cv=kfold, error_score=0, random_state=1, n_jobs=jobs, n_iter=n_iterations)
fig_cols = 2
fig_rows = 4

xgb_means_list = []
xgb_names_list = []
xgb_params_list = []
xgb_best_result_list = []
xgb_max_means_list = []
xgb_gamma = []
xgb_n_estimator = []
xgb_max_depth = []

for i in range(0, int(len(X_list_m))):
    grid_result = grid.fit(X_list_m[i][0], y_list_m[i][0])
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    
    xgb_names = []
    for k in range(len(params)):
        new_string = str(params[k]['max_depth']) + ' ' + str(params[k]['n_estimators']) + ' ' +str(params[k]['gamma'])
        xgb_names.append(new_string)
    
    index_to_rm = []
    for index, item in enumerate(params):
        if item in params[:index]:
            index_to_rm.append(index)
    means = np.delete(means, index_to_rm)
    xgb_names = np.delete(xgb_names, index_to_rm)
    params = np.delete(params, index_to_rm)
    max_mean = np.argmax(means)
    
    xgb_means_list.append(means)
    xgb_names_list.append(xgb_names)
    xgb_params_list.append(params)
    xgb_max_means_list.append(max_mean)
    xgb_best_result_list.append(grid_result.best_score_)
    
    print(f'Best {scoring_metric} of {round(grid_result.best_score_,4)} for max depth of {grid_result.best_params_["max_depth"]}, gamma of {grid_result.best_params_["gamma"]}, \
        and number of estimators of {grid_result.best_params_["n_estimators"]}')
    
    xgb_n_estimator.append(grid_result.best_params_["n_estimators"])
    xgb_max_depth.append(grid_result.best_params_["max_depth"])
    xgb_gamma.append(grid_result.best_params_["gamma"])

#### 6.9.2. With SMOTEENN

In [ ]:
cachedir = mkdtemp()
memory = Memory(cachedir=cachedir, verbose=10)

In [ ]:
params_list_xgb_smoteenn = dict()
params_list_xgb_smoteenn['model__n_estimators'] = Integer(10, 3500)
params_list_xgb_smoteenn['model__max_depth'] = Integer(1, 25)
params_list_xgb_smoteenn['model__gamma'] = Real(0, 0.9)
pipe = Pipeline([('SMOTEENN', SMOTEENN(smote=SMOTE(sampling_strategy='minority', random_state=1), enn=EditedNearestNeighbours(sampling_strategy='all'), 
                                       random_state=1, n_jobs=jobs)), ('model', XGBClassifier(random_state=1, n_jobs=jobs, tree_method='hist'))], memory=memory)
kfold = StratifiedKFold(n_splits=k_splits, random_state=1, shuffle=True)
grid = BayesSearchCV(estimator=pipe, search_spaces=params_list_xgb_smoteenn, scoring=scoring_metric, cv=kfold, error_score=0, random_state=1, n_jobs=jobs, n_iter=n_iterations)
fig_cols = 2
fig_rows = 4

for i in range(0, int(len(X_list_m))):
    grid_result = grid.fit(X_list_m[i][0], y_list_m[i][0])
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    
    xgb_names = []
    for k in range(len(params)):
        new_string = str(params[k]['model__n_estimators']) + ' ' + str(params[k]['model__max_depth']) + ' ' + str(params[k]['model__gamma'])
        xgb_names.append(new_string)
    
    index_to_rm = []
    for index, item in enumerate(params):
        if item in params[:index]:
            index_to_rm.append(index)
    means = np.delete(means, index_to_rm)
    xgb_names = np.delete(xgb_names, index_to_rm)
    params = np.delete(params, index_to_rm)
    max_mean = np.argmax(means)
    
    xgb_means_list.append(means)
    xgb_names_list.append(xgb_names)
    xgb_params_list.append(params)
    xgb_max_means_list.append(max_mean)
    xgb_best_result_list.append(grid_result.best_score_)
    
    print(f'Best {scoring_metric} of {round(grid_result.best_score_,4)} for max_depth of {grid_result.best_params_["model__max_depth"]}, gamma of {grid_result.best_params_["model__gamma"]}, \
        and number of estimators of {grid_result.best_params_["model__n_estimators"]}')
    
    xgb_n_estimator.append(grid_result.best_params_["model__n_estimators"])
    xgb_max_depth.append(grid_result.best_params_["model__max_depth"])
    xgb_gamma.append(grid_result.best_params_["model__gamma"])
    
rmtree(cachedir)

In [ ]:
col = -1
row = 0

fig, ax = plt.subplots(4, 2)
fig.set_size_inches((30, 30))
fig.tight_layout(h_pad=25.0, w_pad=4)
fig.set_dpi(dpi_low)
for i in range(0, 8):
    if i % 2 == 0:
        col += 1
        row = 0
    if i % 2 == 1:
        row += 1
        
    ax[col, row].plot(range(len(xgb_names_list[i])), xgb_means_list[i], color='grey', marker='o', markerfacecolor='black')
    ax[col, row].plot(xgb_max_means_list[i], xgb_best_result_list[i], marker='o', markerfacecolor='red', markeredgecolor="red")
    ax[col, row].set_title(f'Mean {scoring_metric} for XGB with stratified {k_splits}-fold crossvalidation\n{name_list[i]}')
    ax[col, row].set_xticks(range(len(xgb_names_list[i])))
    ax[col, row].set_xticklabels(xgb_names_list[i], rotation=45, ha='right')
    ax[col, row].set_xlabel('XGB Hyperparameters (Max depth and n estimators)')
    ax[col, row].set_ylabel(scoring_metric)

plt.show()

### 6.10. XGB with optimal hyperparameters

In [ ]:
xgb_models = []
xgb_accuracy = []
xgb_precision = []
xgb_recall = []
xgb_f1 = []
xgb_classification_report = []
xgb_test_prediction = []

for i, variant in enumerate(X_list):
    xgb = XGBClassifier(random_state=1, n_jobs=jobs, n_estimators=xgb_n_estimator[i], max_depth=xgb_max_depth[i], gamma=xgb_gamma[i], tree_method='hist')
    xgb.fit(variant[0], y_list[i][0])
    xgb_test_pred = xgb.predict(variant[1])
    xgb_test_prediction.append(xgb_test_pred)

    xgb_accuracy.append(xgb.score(variant[1], y_list[i][1]))
    xgb_precision.append(precision_score(y_list[i][1], xgb_test_pred))
    xgb_recall.append(recall_score(y_list[i][1], xgb_test_pred))
    xgb_f1.append(f1_score(y_list[i][1], xgb_test_pred))

    xgb_classification_report.append(classification_report(y_list[i][1], xgb_test_pred, target_names=label))

    xgb_models.append(xgb)

In [ ]:
xgb_pr = []

fig, ax = plt.subplots(4, 2)
fig.set_size_inches((20, 15))
fig.tight_layout(pad=5.0)
col = -1
row = 0
for i in range(0, 8):
    if i % 2 == 0:
        col += 1
        row = 0
    elif i % 2 == 1:
        row += 1
    
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name='XGB', color='black', ax=ax[col, row])
    xgb_pr.append(xgb_display)
    ax[col, row].set_title('XGB Precision-Recall curve ' + name_list[i], fontweight='bold')
    ax[col, row].legend(loc='best')
    ax[col, row].set_xlabel('Recall')
    ax[col, row].set_ylabel('Precision')
    ax[col, row].set_xlim(-0.05, 1.05)
    ax[col, row].set_xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax[col, row].set_ylim(0, 1.05)
    ax[col, row].set_yticks([0.00, 0.25, 0.50, 0.75, 1.00])
plt.show()

#### Summary

In [ ]:
linestyle_list = ['solid', 'dashed', 'dotted', 'dashdot', 'solid', 'dashed', 'dotted', 'dashdot']
color_list = ['black', 'black', 'black', 'black', 'grey', 'grey', 'grey', 'grey']

ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, int(len(xgb_means_list))):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, 4):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(4, int(len(xgb_means_list))):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

AUROC

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, int(len(xgb_means_list))):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, 4):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(4, int(len(xgb_means_list))):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

Plot confusion matrix for each classifier

In [ ]:
confusion_matrix_list = []
heat_map_list = []

temp_conf_mat = []
temp_heat_map = []
for i in range(0, int(len(xgb_means_list))):
    conf_mat = confusion_matrix(y_list[i][1], xgb_test_prediction[i])
    temp_conf_mat.append(conf_mat)
    heat = pd.DataFrame(conf_mat, columns=np.unique(y_list[i][1]), index=np.unique(y_list[i][1]))
    heat.index.name = 'Actual'
    heat.columns.name = 'Predicted'
    temp_heat_map.append(heat)

confusion_matrix_list.append(temp_conf_mat)
heat_map_list.append(temp_heat_map)

fig, ax = plt.subplots(int(len(xgb_means_list)/4), 4, figsize=(20, 18))
fig.subplots_adjust(left=None, bottom=None, right=2, top=None, wspace=None, hspace=0.3)

sns.set(font_scale=1.4)
lc = 'Grey'
lw = 1
col = -1
row = 0
for i in range(int(len(xgb_means_list))):
    if i % 4 == 0:
        col += 1
        row = 0
    else:
        row += 1
    
    sns.heatmap(heat_map_list[-1][i], cmap='Greys', annot=True, annot_kws={'size': 16}, fmt='g', ax=ax[col, row], xticklabels=label, yticklabels=label, linecolor=lc, linewidths=lw)
    ax[col, row].set_title(('XGB '+name_list[i]), fontweight='bold')
sns.set(font_scale=1)
plt.show()

In [ ]:
matplotlib.rc_file_defaults()

Save output for model persistence

In [ ]:
data = {'Model 1': [], 'Model 2': [], 'Model 3': [], 'Model 4': [], 'Model 5': [], 'Model 6': [], 'Model 7': [], 'Model 8': []}
for i, model in enumerate(data):
    data[model].append(xgb_means_list[i])
    data[model].append(xgb_params_list[i])
    data[model].append(xgb_best_result_list[i])
    data[model].append(xgb_n_estimator[i])
    data[model].append(xgb_max_depth[i])
    data[model].append(xgb_gamma[i])
    dump(xgb_models[i], f'models/output-xgb{i}.joblib')
    data[model].append(xgb_accuracy[i])
    data[model].append(xgb_precision[i])
    data[model].append(xgb_recall[i])
    data[model].append(xgb_f1[i])
    data[model].append(xgb_classification_report[i])
    data[model].append(xgb_test_prediction[i])
    data[model].append(xgb_pr[i])
    data[model].append(heat_map_list[0][i])
    
output_df = pd.DataFrame(data)
index_list = ['xgb_means_list','xgb_params_list','xgb_best_result_list','xgb_best_n_estimator','xgb_best_max_depth','xgb_gamma','xgb_accuracy','xgb_precision','xgb_recall','xgb_f1',
              'xgb_classification_report','xgb_test_prediction','xgb_pr','heat_map_log_reg']
output_df.insert(0, 'Name', index_list)
output_df.set_index(output_df['Name'], inplace=True)
output_df.drop('Name', axis=1, inplace=True)
dump(output_df, 'models/output-xgb.joblib')

### 6.10a. XGB without optimal hyperparameters

In [ ]:
xgb_models = []
xgb_accuracy = []
xgb_precision = []
xgb_recall = []
xgb_f1 = []
xgb_classification_report = []
xgb_test_prediction = []

for i, variant in enumerate(X_list):
    xgb = XGBClassifier(random_state=1, n_jobs=jobs, tree_method='hist')
    xgb.fit(variant[0], y_list[i][0])
    xgb_test_pred = xgb.predict(variant[1])
    xgb_test_prediction.append(xgb_test_pred)

    xgb_accuracy.append(xgb.score(variant[1], y_list[i][1]))
    xgb_precision.append(precision_score(y_list[i][1], xgb_test_pred))
    xgb_recall.append(recall_score(y_list[i][1], xgb_test_pred))
    xgb_f1.append(f1_score(y_list[i][1], xgb_test_pred))

    xgb_classification_report.append(classification_report(y_list[i][1], xgb_test_pred, target_names=label))

    xgb_models.append(xgb)

In [ ]:
xgb_pr = []

fig, ax = plt.subplots(4, 2)
fig.set_size_inches((20, 15))
fig.tight_layout(pad=5.0)
col = -1
row = 0
for i in range(0, 8):
    if i % 2 == 0:
        col += 1
        row = 0
    elif i % 2 == 1:
        row += 1
    
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name='XGB', color='black', ax=ax[col, row])
    xgb_pr.append(xgb_display)
    ax[col, row].set_title('XGB Precision-Recall curve ' + name_list[i], fontweight='bold')
    ax[col, row].legend(loc='best')
    ax[col, row].set_xlabel('Recall')
    ax[col, row].set_ylabel('Precision')
    ax[col, row].set_xlim(-0.05, 1.05)
    ax[col, row].set_xticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
    ax[col, row].set_ylim(0, 1.05)
    ax[col, row].set_yticks([0.00, 0.25, 0.50, 0.75, 1.00])
plt.show()

#### Summary

In [ ]:
linestyle_list = ['solid', 'dashed', 'dotted', 'dashdot', 'solid', 'dashed', 'dotted', 'dashdot']
color_list = ['black', 'black', 'black', 'black', 'grey', 'grey', 'grey', 'grey']

ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, int(len(xgb_means_list))):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, 4):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(4, int(len(xgb_means_list))):
    xgb_display = PrecisionRecallDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined Precision-Recall curve")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

AUROC

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, int(len(xgb_means_list))):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(0, 4):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

In [ ]:
ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(8, 4)
fig.set_dpi(dpi_high)
for i in range(4, int(len(xgb_means_list))):
    xgb_display = RocCurveDisplay.from_estimator(xgb_models[i], X_list[i][1], y_list[i][1], name=f'XGB {name_list[i]}', 
                                                            color=color_list[i], linestyle=linestyle_list[i], ax=plt.gca())
ax.set_title("XGB Combined AUROC")
ax.legend(loc='best', prop={'size': 8})
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 1.05)
plt.show()

Plot confusion matrix for each classifier

In [ ]:
confusion_matrix_list = []
heat_map_list = []

temp_conf_mat = []
temp_heat_map = []
for i in range(0, int(len(xgb_means_list))):
    conf_mat = confusion_matrix(y_list[i][1], xgb_test_prediction[i])
    temp_conf_mat.append(conf_mat)
    heat = pd.DataFrame(conf_mat, columns=np.unique(y_list[i][1]), index=np.unique(y_list[i][1]))
    heat.index.name = 'Actual'
    heat.columns.name = 'Predicted'
    temp_heat_map.append(heat)

confusion_matrix_list.append(temp_conf_mat)
heat_map_list.append(temp_heat_map)

fig, ax = plt.subplots(int(len(xgb_means_list)/4), 4, figsize=(20, 18))
fig.subplots_adjust(left=None, bottom=None, right=2, top=None, wspace=None, hspace=0.3)

sns.set(font_scale=1.4)
lc = 'Grey'
lw = 1
col = -1
row = 0
for i in range(int(len(xgb_means_list))):
    if i % 4 == 0:
        col += 1
        row = 0
    else:
        row += 1
    
    sns.heatmap(heat_map_list[-1][i], cmap='Greys', annot=True, annot_kws={'size': 16}, fmt='g', ax=ax[col, row], xticklabels=label, yticklabels=label, linecolor=lc, linewidths=lw)
    ax[col, row].set_title(('XGB '+name_list[i]), fontweight='bold')
sns.set(font_scale=1)
plt.show()

In [ ]:
matplotlib.rc_file_defaults()

Save output for model persistence

In [ ]:
data = {'Model 1': [], 'Model 2': [], 'Model 3': [], 'Model 4': [], 'Model 5': [], 'Model 6': [], 'Model 7': [], 'Model 8': []}
for i, model in enumerate(data):
    data[model].append(xgb_means_list[i])
    data[model].append(xgb_params_list[i])
    data[model].append(xgb_best_result_list[i])
    data[model].append(xgb_n_estimator[i])
    data[model].append(xgb_max_depth[i])
    dump(xgb_models[i], f'models/output-xgb{i}-no-hpt.joblib')
    data[model].append(xgb_accuracy[i])
    data[model].append(xgb_precision[i])
    data[model].append(xgb_recall[i])
    data[model].append(xgb_f1[i])
    data[model].append(xgb_classification_report[i])
    data[model].append(xgb_test_prediction[i])
    data[model].append(xgb_pr[i])
    data[model].append(heat_map_list[0][i])
    
output_df = pd.DataFrame(data)
index_list = ['xgb_means_list','xgb_params_list','xgb_best_result_list','xgb_best_n_estimator','xgb_best_max_depth','xgb_accuracy','xgb_precision','xgb_recall','xgb_f1','xgb_classification_report',
                'xgb_test_prediction','xgb_pr','heat_map_log_reg']
output_df.insert(0, 'Name', index_list)
output_df.set_index(output_df['Name'], inplace=True)
output_df.drop('Name', axis=1, inplace=True)
dump(output_df, 'models/output-xgb-no-hpt.joblib')